In [1]:
#*******************************************************************************************
 #
 #  File Name:  charity_binning_optimization.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, charity_binning_optimization.ipynb,
 #      reads a csv file, charity_data.csv, in the folder, resources, and uses Python
 #      and deep learning methods to determine the optimal model hyperparameters.
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  12/02/2023      Initial Development                     Nicholas J. George
 #
 #******************************************************************************************/

import logx
logx.create_directory(logx.MODELS_DIRECTORY_PATH)

import pandasx

import deep_learningx
import charity_constants

import copy
import pickle
import datetime
import io
import json
import time

import pandas as pd
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

pd.options.mode.chained_assignment = None

2024-05-06 10:20:59.960693: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CONSTANT_LOCAL_FILE_NAME = 'charity_binning_optimization.ipynb'


logx.set_log_mode(False)

logx.set_image_mode(False)


logx.begin_program('charity_binning_optimization')

Program execution begins...



# <br> **Section 1: Extraction**

## **1.1: Read the CSV data into a Pandas DataFrame**

In [3]:
data_type_dictionary \
    = {'EIN': str,
       'NAME':	str,
       'APPLICATION_TYPE': str,
       'AFFILIATION': str,
       'CLASSIFICATION': str,
       'USE_CASE': str,
       'ORGANIZATION': str,
       'STATUS': int,
       'INCOME_AMT': str,
       'SPECIAL_CONSIDERATIONS': str,
       'ASK_AMT': int,
       'IS_SUCCESSFUL': int}

charity_dataframe \
    = pd.read_csv(charity_constants.INPUT_FILE_PATH, dtype = data_type_dictionary)

logx.log_write_object(charity_dataframe)

## **1.2: Display Charity DataFrame**

In [4]:
pandasx.return_formatted_table(charity_dataframe, 'Table 1.2: Charity Data Table')

EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
10566033,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
10570430,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


# <br> **Section 2: Preprocessing**

## **2.1: Number of Records**

In [5]:
logx.print_and_log_text \
    ('\033[1m'
     + 'There are {:,} student loans '.format(len(charity_dataframe))
     + 'in this data set.\n\n' \
     + '\033[0m')

There are 34,299 student loans in this data set.




## **2.2: Column Counts**

In [6]:
logx.print_and_log_text('\033[1m' + str(charity_dataframe.count()) + '\033[0m')

EIN                       34299
NAME                      34299
APPLICATION_TYPE          34299
AFFILIATION               34299
CLASSIFICATION            34299
USE_CASE                  34299
ORGANIZATION              34299
STATUS                    34299
INCOME_AMT                34299
SPECIAL_CONSIDERATIONS    34299
ASK_AMT                   34299
IS_SUCCESSFUL             34299
dtype: int64


## **2.3: Unique Counts**

In [7]:
pandasx.display_dataframe_column_counts(charity_dataframe)

EIN: 34,299

NAME: 19,568

APPLICATION_TYPE: 17

AFFILIATION: 6

CLASSIFICATION: 71

USE_CASE: 5

ORGANIZATION: 4

STATUS: 2

INCOME_AMT: 9

SPECIAL_CONSIDERATIONS: 2

ASK_AMT: 8,747

IS_SUCCESSFUL: 2



## **2.4: Value Counts**

### **APPLICATION_TYPE**

In [8]:
charity_dataframe['APPLICATION_TYPE'].value_counts().sort_values(ascending = False)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

### **AFFILIATION**

In [9]:
charity_dataframe['AFFILIATION'].value_counts().sort_values(ascending = False)

AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

### **CLASSIFICATION**

In [10]:
charity_dataframe['CLASSIFICATION'].value_counts().sort_values(ascending = False)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: count, Length: 71, dtype: int64

### **USE_CASE**

In [11]:
charity_dataframe['USE_CASE'].value_counts().sort_values(ascending = False)

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64

### **ORGANIZATION**

In [12]:
charity_dataframe['ORGANIZATION'].value_counts().sort_values(ascending = False)

ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: count, dtype: int64

### **STATUS**

In [13]:
charity_dataframe['STATUS'].value_counts().sort_values(ascending = False)

STATUS
1    34294
0        5
Name: count, dtype: int64

### **INCOME_AMT**

In [14]:
charity_dataframe['INCOME_AMT'].value_counts().sort_values(ascending = False)

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

### **SPECIAL_CONSIDERATIONS**

In [15]:
charity_dataframe['SPECIAL_CONSIDERATIONS'].value_counts().sort_values(ascending = False)

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

### **ASK_AMT**

In [16]:
charity_dataframe['ASK_AMT'].value_counts().sort_values(ascending = False)

ASK_AMT
5000        25398
15583           3
63981           3
6725            3
10478           3
            ...  
11336           1
1270766         1
121619          1
46915           1
36500179        1
Name: count, Length: 8747, dtype: int64

## **2.5: Change ASK_AMT Integers to Categories**

In [17]:
updated_charity_dataframe = charity_dataframe.rename(columns = {'ASK_AMT': 'TEMP'})

logx.log_write_object(updated_charity_dataframe)

In [18]:
ask_amt_bins_integer_list \
    = [0, 10000, 25000, 100000, 500000, 1000000, 5000000, 10000000, 50000000, 100000000]

ask_amt_labels_string_list \
    = ['0-9999', '10000-24999', '25000-99999', '100000-499999', '500000-999999', '1M-5M', '5M-10M', '10M-50M', '50M+']

In [19]:
updated_charity_dataframe['ASK_AMT'] \
    = pd.cut \
        (x = updated_charity_dataframe['TEMP'],
         bins = ask_amt_bins_integer_list,
         labels = ask_amt_labels_string_list)

updated_charity_dataframe \
    = updated_charity_dataframe \
        [['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
          'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT',
          'IS_SUCCESSFUL', 'TEMP']]

logx.log_write_object(updated_charity_dataframe['ASK_AMT'])

In [20]:
updated_charity_dataframe['ASK_AMT'].value_counts().sort_values(ascending = False)

ASK_AMT
0-9999           25946
25000-99999       2685
100000-499999     2305
10000-24999       1135
1M-5M              940
500000-999999      651
10M-50M            258
5M-10M             225
50M+                53
Name: count, dtype: int64

## **2.6: Drop the Non-Beneficial ID Columns**

In [21]:
drop_columns_string_list = ['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'TEMP']

if updated_charity_dataframe.columns.isin(drop_columns_string_list).any():

    updated_charity_dataframe.drop(drop_columns_string_list, axis = 1, inplace = True)

logx.log_write_object(updated_charity_dataframe)

## **2.7: Convert All Features to Uppercase**

In [22]:
updated_charity_dataframe['NAME'] = updated_charity_dataframe['NAME'].str.upper()

updated_charity_dataframe['APPLICATION_TYPE'] = updated_charity_dataframe['APPLICATION_TYPE'].str.upper()

updated_charity_dataframe['AFFILIATION'] = updated_charity_dataframe['AFFILIATION'].str.upper()

updated_charity_dataframe['CLASSIFICATION'] = updated_charity_dataframe['CLASSIFICATION'].str.upper()

updated_charity_dataframe['USE_CASE'] = updated_charity_dataframe['USE_CASE'].str.upper()

updated_charity_dataframe['ORGANIZATION'] = updated_charity_dataframe['ORGANIZATION'].str.upper()

updated_charity_dataframe['INCOME_AMT'] = updated_charity_dataframe['INCOME_AMT'].str.upper()

updated_charity_dataframe['ASK_AMT'] = updated_charity_dataframe['ASK_AMT'].str.upper()

logx.log_write_object(updated_charity_dataframe)

## **2.8: Display Updated DataFrame**

In [23]:
pandasx.return_formatted_table(updated_charity_dataframe, 'Table 2.8: Updated Charity Data Table')

NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
BLUE KNIGHTS MOTORCYCLE CLUB,T10,INDEPENDENT,C1000,PRODUCTDEV,ASSOCIATION,0,0-9999,1
AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,INDEPENDENT,C2000,PRESERVATION,CO-OPERATIVE,1-9999,100000-499999,1
ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,COMPANYSPONSORED,C3000,PRODUCTDEV,ASSOCIATION,0,0-9999,0
SOUTHSIDE ATHLETIC ASSOCIATION,T3,COMPANYSPONSORED,C2000,PRESERVATION,TRUST,10000-24999,0-9999,1
GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,INDEPENDENT,C1000,HEATHCARE,TRUST,100000-499999,100000-499999,1
MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,INDEPENDENT,C1200,PRESERVATION,TRUST,0,0-9999,1
FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,INDEPENDENT,C1000,PRESERVATION,TRUST,100000-499999,25000-99999,1
ISRAEL EMERGENCY ALLIANCE,T3,INDEPENDENT,C2000,PRESERVATION,TRUST,10M-50M,5M-10M,1
ARAMCO BRATS INC,T7,INDEPENDENT,C1000,PRODUCTDEV,TRUST,1-9999,25000-99999,1
INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,COMPANYSPONSORED,C3000,PRODUCTDEV,ASSOCIATION,0,0-9999,0


# <br> **Section 3: Find Optimal Binning Values**

## **3.1: Set Dictionaries**

In [24]:
optimal_bins_dictionary \
    = {'NAME': {'bin': 0, 'accuracy': 0.0},
       'APPLICATION_TYPE': {'bin': 0, 'accuracy': 0.0},
       'AFFILIATION': {'bin': 0, 'accuracy': 0.0},
       'CLASSIFICATION': {'bin': 0, 'accuracy': 0.0},
       'USE_CASE': {'bin': 0, 'accuracy': 0.0},
       'ORGANIZATION': {'bin': 0, 'accuracy': 0.0},
       'INCOME_AMT': {'bin': 0, 'accuracy': 0.0},
       'ASK_AMT': {'bin': 0, 'accuracy': 0.0}}

In [25]:
binning_parameters_dictionary \
    = {'trials': 10,
       'replacement': 'Other',
       'target': 'IS_SUCCESSFUL',
       'random_state': 21,
       'test_size': 0.2,
       'first_layer_units': 53,
       'first_activation': 'relu',
       'hidden_layer_units': 20,
       'hidden_activation': 'relu',
       'output_layer_units': 1,
       'output_activation': 'sigmoid',
       'loss': 'log_cosh',
       'optimizer': 'adamax',
       'metrics': 'accuracy',
       'monitor': 'val_accuracy',
       'mode': 'max',
       'patience': 42,
       'restore_best_weights': True,
       'epochs': 1000,
       'verbose': 2}

deep_learningx.set_binning_parameters_dictionary(binning_parameters_dictionary)

## **3.2: Examine Each Column for Optimal Binning Values**

### **NAME**

In [26]:
updated_charity_dataframe['NAME'], optimal_bins_dictionary['NAME'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'NAME', 'OTHER', 0.0)

logx.log_write_object \
    (updated_charity_dataframe['NAME'] \
        .value_counts().sort_values(ascending = False))

Processing: 93/93

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7085 - loss: 0.0965 - val_accuracy: 0.7673 - val_loss: 0.0745
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7790 - loss: 0.0731 - val_accuracy: 0.7968 - val_loss: 0.0679
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7996 - loss: 0.0662 - val_accuracy: 0.8000 - val_loss: 0.0660
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8197 - loss: 0.0602 - val_accuracy: 0.8023 - val_loss: 0.0649
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8375 - loss: 0.0543 - val_accuracy: 0.8003 - val_loss: 0.0645
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8672 - loss: 0.0476 - val_accuracy: 0.8009 - val_loss: 0.0664
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accu

### **APPLICATION_TYPE**

In [27]:
updated_charity_dataframe['APPLICATION_TYPE'], optimal_bins_dictionary['APPLICATION_TYPE'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'APPLICATION_TYPE', 'T20', 
         optimal_bins_dictionary['NAME']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['APPLICATION_TYPE'].value_counts().sort_values(ascending = False))

Processing: 16/16

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step - accuracy: 0.6975 - loss: 0.0989 - val_accuracy: 0.7593 - val_loss: 0.0774
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 833us/step - accuracy: 0.7624 - loss: 0.0774 - val_accuracy: 0.7888 - val_loss: 0.0710
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 863us/step - accuracy: 0.7850 - loss: 0.0719 - val_accuracy: 0.8013 - val_loss: 0.0683
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 838us/step - accuracy: 0.7903 - loss: 0.0694 - val_accuracy: 0.8004 - val_loss: 0.0667
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 847us/step - accuracy: 0.7972 - loss: 0.0674 - val_accuracy: 0.8006 - val_loss: 0.0657
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 851us/step - accuracy: 0.8012 - loss: 0.0661 - val_accuracy: 0.8038 - val_loss: 0.0651
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 864

### **AFFILIATION**

In [28]:
updated_charity_dataframe['AFFILIATION'], optimal_bins_dictionary['AFFILIATION'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'AFFILIATION', 'OTHER', 
         optimal_bins_dictionary['APPLICATION_TYPE']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['AFFILIATION'].value_counts().sort_values(ascending = False))

Processing: 7/7

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 903us/step - accuracy: 0.6888 - loss: 0.1004 - val_accuracy: 0.7676 - val_loss: 0.0778
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - accuracy: 0.7709 - loss: 0.0754 - val_accuracy: 0.7913 - val_loss: 0.0710
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - accuracy: 0.7863 - loss: 0.0709 - val_accuracy: 0.7968 - val_loss: 0.0680
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 797us/step - accuracy: 0.7931 - loss: 0.0679 - val_accuracy: 0.7994 - val_loss: 0.0663
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.7949 - loss: 0.0674 - val_accuracy: 0.8022 - val_loss: 0.0659
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 815us/step - accuracy: 0.7999 - loss: 0.0659 - val_accuracy: 0.8067 - val_loss: 0.0650
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 812us

### **CLASSIFICATION**

In [29]:
updated_charity_dataframe['CLASSIFICATION'], optimal_bins_dictionary['CLASSIFICATION'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'CLASSIFICATION', 'C2200', 
         optimal_bins_dictionary['AFFILIATION']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['CLASSIFICATION'].value_counts().sort_values(ascending = False))

Processing: 34/34

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 927us/step - accuracy: 0.6713 - loss: 0.1017 - val_accuracy: 0.7590 - val_loss: 0.0804
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 806us/step - accuracy: 0.7603 - loss: 0.0785 - val_accuracy: 0.7891 - val_loss: 0.0718
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - accuracy: 0.7816 - loss: 0.0718 - val_accuracy: 0.7937 - val_loss: 0.0685
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 796us/step - accuracy: 0.7932 - loss: 0.0686 - val_accuracy: 0.7966 - val_loss: 0.0669
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 796us/step - accuracy: 0.7984 - loss: 0.0668 - val_accuracy: 0.8038 - val_loss: 0.0657
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 795us/step - accuracy: 0.7996 - loss: 0.0663 - val_accuracy: 0.8015 - val_loss: 0.0652
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 796

### **USE_CASE**

In [30]:
updated_charity_dataframe['USE_CASE'], optimal_bins_dictionary['USE_CASE'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'USE_CASE', 'OTHER',
         optimal_bins_dictionary['CLASSIFICATION']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['USE_CASE'].value_counts().sort_values(ascending = False))

Processing: 6/6

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 895us/step - accuracy: 0.6599 - loss: 0.1027 - val_accuracy: 0.7587 - val_loss: 0.0776
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - accuracy: 0.7648 - loss: 0.0767 - val_accuracy: 0.7835 - val_loss: 0.0716
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - accuracy: 0.7867 - loss: 0.0706 - val_accuracy: 0.7948 - val_loss: 0.0681
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.7947 - loss: 0.0679 - val_accuracy: 0.8009 - val_loss: 0.0664
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 781us/step - accuracy: 0.7996 - loss: 0.0661 - val_accuracy: 0.8023 - val_loss: 0.0655
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 0.8027 - loss: 0.0651 - val_accuracy: 0.8038 - val_loss: 0.0649
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 779us

### **ORGANIZATION**

In [31]:
updated_charity_dataframe['ORGANIZATION'], optimal_bins_dictionary['ORGANIZATION'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'ORGANIZATION', 'OTHER',
         optimal_bins_dictionary['USE_CASE']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['ORGANIZATION'].value_counts().sort_values(ascending = False))

Processing: 5/5

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 956us/step - accuracy: 0.6931 - loss: 0.0993 - val_accuracy: 0.7528 - val_loss: 0.0791
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step - accuracy: 0.7577 - loss: 0.0787 - val_accuracy: 0.7897 - val_loss: 0.0713
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step - accuracy: 0.7838 - loss: 0.0716 - val_accuracy: 0.7914 - val_loss: 0.0691
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.7947 - loss: 0.0685 - val_accuracy: 0.7971 - val_loss: 0.0668
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step - accuracy: 0.7967 - loss: 0.0670 - val_accuracy: 0.7991 - val_loss: 0.0658
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 777us/step - accuracy: 0.8026 - loss: 0.0652 - val_accuracy: 0.8023 - val_loss: 0.0654
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 779us

### **INCOME_AMT**

In [32]:
updated_charity_dataframe['INCOME_AMT'], optimal_bins_dictionary['INCOME_AMT'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'INCOME_AMT', 'OTHER',
         optimal_bins_dictionary['ORGANIZATION']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['INCOME_AMT'].value_counts().sort_values(ascending = False))

Processing: 10/10

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 904us/step - accuracy: 0.6884 - loss: 0.1024 - val_accuracy: 0.7558 - val_loss: 0.0785
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 0.7608 - loss: 0.0772 - val_accuracy: 0.7873 - val_loss: 0.0712
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - accuracy: 0.7875 - loss: 0.0707 - val_accuracy: 0.7952 - val_loss: 0.0683
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step - accuracy: 0.7944 - loss: 0.0681 - val_accuracy: 0.8009 - val_loss: 0.0665
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 765us/step - accuracy: 0.8001 - loss: 0.0662 - val_accuracy: 0.8020 - val_loss: 0.0655
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.8028 - loss: 0.0649 - val_accuracy: 0.8063 - val_loss: 0.0647
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 761

### **ASK_AMT**

In [33]:
updated_charity_dataframe['ASK_AMT'], optimal_bins_dictionary['ASK_AMT'] \
    = deep_learningx.return_optimal_binning_value \
        (updated_charity_dataframe, 'ASK_AMT', 'OTHER',
         optimal_bins_dictionary['INCOME_AMT']['accuracy'])

logx.log_write_object \
    (updated_charity_dataframe['ASK_AMT'].value_counts().sort_values(ascending = False))

Processing: 10/10

Replacing values...

Setting dummy values...

Splitting data into training and testing...

Creating model...

Compiling...

Fitting data...

Epoch 1/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 893us/step - accuracy: 0.6799 - loss: 0.1020 - val_accuracy: 0.7541 - val_loss: 0.0801
Epoch 2/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - accuracy: 0.7534 - loss: 0.0794 - val_accuracy: 0.7854 - val_loss: 0.0717
Epoch 3/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.7867 - loss: 0.0713 - val_accuracy: 0.7915 - val_loss: 0.0694
Epoch 4/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 763us/step - accuracy: 0.7917 - loss: 0.0693 - val_accuracy: 0.7940 - val_loss: 0.0669
Epoch 5/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 762us/step - accuracy: 0.7983 - loss: 0.0667 - val_accuracy: 0.7968 - val_loss: 0.0660
Epoch 6/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.7956 - loss: 0.0667 - val_accuracy: 0.7996 - val_loss: 0.0657
Epoch 7/1000
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 763

## **3.3: Display Optimal Bins Dictionary**

In [34]:
for ele in optimal_bins_dictionary:

    logx.print_and_log_text('\033[1m' + f"{ele}:  {optimal_bins_dictionary[ele]}\n" + '\033[0m')

NAME:  {'bin': 1, 'accuracy': 0.8097667694091797}

APPLICATION_TYPE:  {'bin': 1216, 'accuracy': 0.8119533658027649}

AFFILIATION:  {'bin': 33, 'accuracy': 0.812828004360199}

CLASSIFICATION:  {'bin': 116, 'accuracy': 0.8135568499565125}

USE_CASE:  {'bin': 3, 'accuracy': 0.8144314885139465}

ORGANIZATION:  {'bin': 0, 'accuracy': 0.8122448921203613}

INCOME_AMT:  {'bin': 955, 'accuracy': 0.8126822113990784}

ASK_AMT:  {'bin': 940, 'accuracy': 0.812828004360199}



## **3.3: Export Optimal Bins Dictionary**

In [35]:
pickle.dump \
    (optimal_bins_dictionary,
     open(charity_constants.BINNING_PARAMETERS_FILE_PATH, 'wb'))

In [36]:
pickle.dump \
    (updated_charity_dataframe,
     open(charity_constants.BINNED_CHARITY_DATA_FILE_PATH, 'wb'))

In [37]:
# logx.end_program()